In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from tqdm import tqdm

In [3]:
import malaya

In [4]:
# model = malaya.dependency.transformer(model = 'xlnet', quantized = True)

In [5]:
with open('filtered-dumping-wiki.txt') as fopen:
    data = list(filter(None, fopen.read().split('\n')))
    
data = [i for i in data if len(i) >= 2]

len(data)

2037249

In [6]:
tokenizer = malaya.preprocessing.TOKENIZER().tokenize
sastrawi = malaya.stem.sastrawi()

In [9]:
import copy
import re
from malaya.text.regex import _expressions

start = ['be', 'ber', 'ter', 'se']

def check_bilangan(word):
    if re.findall(_expressions['hypen'], word.lower()):
        stemmed = sastrawi.stem(word)
        splitted = word.split('-')
        for s in start:
            if word.startswith(s) and f'{s}{stemmed}' == splitted[0] and stemmed == splitted[1]:
                return True
    return False

In [11]:
check_bilangan('ayam-ayam')

False

In [17]:
def reset_t(tokens):
    t = []
    for i in range(len(tokens)):
        t.append([tokens[i], 2])
    return t

# [kata bilangan] - [kata bilangan] -> [kata bilangan]
# berjuta-juta -> berjuta
def augment_8_0(t, row):
    text, tokens, tokens_lower = row
    for i in range(len(tokens)):
        if check_bilangan(tokens[i]):
            t[i][0] = tokens[i].split('-')[0]
            t[i][1] = 8

In [18]:
results = []
for text in tqdm(['berjuta-juta rakyat malaysia']):
    tokens = tokenizer(text)
    t = reset_t(tokens)
    t_ = copy.deepcopy(t)
    tokens_lower = tokenizer(text.lower())
    r = (t, tokens, tokens_lower)
    augment_8_0(t_, r)
    results.append((t, t_))

100%|██████████| 1/1 [00:00<00:00, 1478.95it/s]


In [19]:
results

[([['berjuta-juta', 2], ['rakyat', 2], ['malaysia', 2]],
  [['berjuta', 8], ['rakyat', 2], ['malaysia', 2]])]